# Historical Introduction (Apostol, Calculus Vol. 1)

## Scope
This notebook follows the main themes under "Historical Introduction" and turns them into small computational/visual experiments and structured notes.

## Topics
1. The two basic concepts of calculus
2. Historical background
3. The method of exhaustion for the area of a parabolic segment
4. Exercises (referenced by number only)
5. A critical analysis of Archimedes’ method
6. The approach to calculus used in this book


# Two geometric problems that motivate calculus

A useful way to motivate calculus is to start from two closely related geometric questions.  
First, given a curve $y=f(x)$ drawn above a horizontal baseline and an interval $[a,b]$, we want to associate a single number with the region bounded by the curve, the baseline, and the vertical lines $x=a$ and $x=b$. This motivates the notion of area and leads naturally to the idea of an integral. Second, given a point $x=c$ on the same curve, we want to associate a single number with the “steepness” of the tangent line at that point. This motivates the notion of instantaneous rate of change and leads to the idea of a derivative. In the formal development, these two measurements—area and steepness—become the organizing problems around which integral and differential calculus are built (Adapted from the motivating discussion in Apostol, *Calculus*, Vol. 1, 2nd ed., p. 2.).

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from ipywidgets import FloatSlider, VBox, HBox, Output, Layout
from IPython.display import display, clear_output

# --- Görsel ayarlar (okunaklı serif) ---
plt.rcParams.update({
    "font.family": "STIXGeneral",
    "mathtext.fontset": "stix",
    "font.size": 22,
})

out = Output()


# ------------------------------------------------------------
# Temel fonksiyonlar
# ------------------------------------------------------------
def f_curve(x, A=0.9, w=1.0, phi=0.0, shift=1.6):
    """Yukarı kaydırılmış sinüs eğrisi."""
    return shift + A * np.sin(w * x + phi)


def deriv_central(f, x0, h=1e-4):
    """Merkezi fark ile türev yaklaşımı."""
    return (f(x0 + h) - f(x0 - h)) / (2 * h)


def clamp(x, lo, hi):
    return lo if x < lo else hi if x > hi else x


def trapz_area(y, x):
    """NumPy sürüm farkları için trapez entegrali."""
    if hasattr(np, "trapezoid"):
        return np.trapezoid(y, x)
    return np.trapz(y, x)


# ------------------------------------------------------------
# Çizim
# ------------------------------------------------------------
def render(
    A=0.9, w=1.0, phi=0.0, shift=1.6,
    a=1.0, b=6.0, c=7.2,
    xmin=0.0, xmax=10.0,
    fig_path="assets/figures/figure1.png"
):
    # Aralıkları toparla
    a_, b_ = (a, b) if a <= b else (b, a)
    a_ = clamp(a_, xmin, xmax)
    b_ = clamp(b_, xmin, xmax)
    c_ = clamp(c, xmin, xmax)

    # Örnekleme
    N = 2500
    x = np.linspace(xmin, xmax, N)

    f = lambda t: f_curve(t, A=A, w=w, phi=phi, shift=shift)
    y = f(x)

    # Alan (a,b)
    x_area = np.linspace(a_, b_, 15000)
    area = trapz_area(f(x_area), x_area)

    # Teğet (c)
    m = deriv_central(f, c_)
    yc = f(c_)
    tangent = yc + m * (x - c_)

    with out:
        clear_output(wait=True)

        fig, ax = plt.subplots(figsize=(12.5, 8.0), dpi=150)

        # Minimal eksen stili
        for s in ["top", "right", "bottom", "left"]:
            ax.spines[s].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])

        # Eksen okları
        y_top = float(np.max(y) * 1.12)
        ax.annotate("", xy=(xmax, 0), xytext=(xmin, 0),
                    arrowprops=dict(arrowstyle="->", lw=1.9))
        ax.annotate("", xy=(0, y_top), xytext=(0, 0),
                    arrowprops=dict(arrowstyle="->", lw=1.9))

        ax.text(xmax, -0.20, r"$x$", ha="right", va="top", fontsize=26)
        ax.text(-0.20, y_top, r"$y$", ha="right", va="bottom", fontsize=26)

        # Eğri
        ax.plot(x, y, lw=3.2)
        xC = xmin + 0.48 * (xmax - xmin)
        ax.text(xC, f(xC) + 0.20, r"$C$", fontsize=28)

        # Alan gölgesi
        mask = (x >= a_) & (x <= b_)
        ax.fill_between(x[mask], 0, y[mask], alpha=0.18)
        ax.vlines([a_, b_], [0, 0], [f(a_), f(b_)], lw=2.2)

        # Teğet
        ax.plot(x, tangent, lw=2.6)

        # Teğet etiketi (çakışmasın diye biraz kaydır)
        x_lab = clamp(c_ + 0.9, xmin + 0.5, xmax - 1.2)
        y_lab = yc + m * (x_lab - c_)
        ax.text(x_lab, y_lab + 0.18, "Line tangent to $C$", fontsize=18, ha="left")

        # Teğet noktası + yön
        ax.scatter([c_], [yc], s=90, zorder=3)
        dx = 0.9
        ax.annotate("", xy=(c_ + dx, yc + m * dx), xytext=(c_, yc),
                    arrowprops=dict(arrowstyle="->", lw=1.7))
        ax.text(c_ + dx + 0.18, yc + m * dx + 0.04, "steepness", fontsize=18)

        # a, b, c işaretleri
        ax.text(a_, -0.34, r"$a$", ha="center", va="top", fontsize=24)
        ax.text(b_, -0.34, r"$b$", ha="center", va="top", fontsize=24)
        ax.text(c_, -0.34, r"$c$", ha="center", va="top", fontsize=24)

        ax.set_xlim(xmin, xmax)
        ax.set_ylim(-0.60, float(np.max(y) * 1.18))

        # Bilgi kutusu
        info = (
            r"$A(a,b)\approx \int_a^b f(x)\,dx \approx$ " + f"{area:.4f}\n"
            r"$m(c)\approx f'(c)\approx$ " + f"{m:.4f}\n\n"
            "Apostol, Calculus Vol. 1 (2nd ed.), p. 2"
        )
        ax.text(0.06, 0.92, info, transform=ax.transAxes,
                ha="left", va="top", fontsize=17,
                bbox=dict(boxstyle="round,pad=0.55", fc="0.98", ec="0.35", lw=1.0))

        # Başlık/caption
        caption = (
            "Figure 1. Two basic measurements: "
            r"$A(a,b)=\int_a^b f(x)\,dx$ and $m(c)=f'(c)$. "
            "Apostol (Vol. 1, 2nd ed.), p. 2."
        )
        fig.subplots_adjust(bottom=0.20)
        fig.text(0.07, 0.04, caption, fontsize=18)

        # Kaydet
        os.makedirs(os.path.dirname(fig_path), exist_ok=True)
        fig.savefig(fig_path, dpi=300, bbox_inches="tight")
        plt.show()

        # Tablo
        df = pd.DataFrame({
            "Parameter": [
                "A (amplitude)", "w (frequency)", "phi (phase)", "shift",
                "a", "b", "c",
                "Area  A(a,b)", "Slope m(c)"
            ],
            "Value": [
                A, w, phi, shift,
                a_, b_, c_,
                area, m
            ]
        })
        df["Value"] = df["Value"].astype(float).round(6)
        display(df)


# ------------------------------------------------------------
# Slider'lar
# ------------------------------------------------------------
slider_layout = Layout(width="260px")

sA     = FloatSlider(value=0.9, min=0.2, max=1.6, step=0.05, description="A",
                     continuous_update=False, layout=slider_layout)
sw     = FloatSlider(value=1.0, min=0.4, max=2.0, step=0.05, description="w",
                     continuous_update=False, layout=slider_layout)
sphi   = FloatSlider(value=0.0, min=-3.14, max=3.14, step=0.05, description="phi",
                     continuous_update=False, layout=slider_layout)
sshift = FloatSlider(value=1.6, min=0.6, max=3.0, step=0.05, description="shift",
                     continuous_update=False, layout=slider_layout)

sa = FloatSlider(value=1.0, min=0.0, max=9.0, step=0.1, description="a",
                 continuous_update=False, layout=slider_layout)
sb = FloatSlider(value=6.0, min=0.0, max=9.0, step=0.1, description="b",
                 continuous_update=False, layout=slider_layout)
sc = FloatSlider(value=7.2, min=0.0, max=10.0, step=0.1, description="c",
                 continuous_update=False, layout=slider_layout)

controls1 = HBox([sA, sw, sphi, sshift])
controls2 = HBox([sa, sb, sc])

def _update(*args):
    render(
        A=sA.value, w=sw.value, phi=sphi.value, shift=sshift.value,
        a=sa.value, b=sb.value, c=sc.value,
        fig_path="assets/figures/figure1.png"
    )

for wdg in [sA, sw, sphi, sshift, sa, sb, sc]:
    wdg.observe(_update, names="value")

display(VBox([controls1, controls2, out]))
_update()



# Historical background
Calculus, at its core, is concerned with the precise formulation and solution of two fundamental geometric measurement problems: the assignment of a number to represent the area of a region and the assignment of a number to represent the steepness of a tangent line at a point on a curve. In modern language, these two measurements are expressed by the integral and the derivative, which allow us to define the concepts of area and tangent line and to compute, respectively, the area of a given region and the slope of a given tangent. The historical origin of integral calculus goes back more than two thousand years to the Greek method of exhaustion, in which one approximates a given region by inscribed polygonal regions whose areas can be computed exactly, then improves the approximation by using polygons with more and more sides in an attempt to “exhaust” the region. This idea was used with remarkable success by Archimedes to obtain exact formulas for the area of the circle and other special figures, but further development was impossible at the time due to the absence of a suitable algebraic language. The elementary algebra familiar today did not exist in antiquity, and without efficient symbolic notation it was practically impossible to extend the method to general classes of regions. A slow but revolutionary transformation began in the sixteenth century with the replacement of Roman numerals by Hindu–Arabic numerals, the introduction of the symbols + and −, and the growing acceptance of decimal notation; during the same period, the work of Tartaglia, Cardano, and Ferrari on cubic and quartic equations greatly advanced algebraic methods and encouraged the adoption of a more powerful symbolic language. With the widespread use of well-chosen algebraic symbols, interest in the ancient method of exhaustion was revived, leading to many partial results by Cavalieri, Torricelli, Roberval, Fermat, Pascal, and Wallis, and gradually the method evolved into what is now called integral calculus, a new and powerful discipline with applications far beyond geometry. Its greatest impetus came in the seventeenth century through the work of Isaac Newton and Gottfried Wilhelm Leibniz, and its rigorous foundations were established in the nineteenth century by mathematicians such as Augustin-Louis Cauchy and Bernhard Riemann, while further refinements and extensions of the theory continue in contemporary mathematics.

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from ipywidgets import IntSlider, FloatSlider, Play, HBox, VBox, Output, Layout, Checkbox
from IPython.display import display, clear_output, Markdown

# ------------------------------------------------------------
# Style
# ------------------------------------------------------------
plt.rcParams.update({
    "font.family": "STIXGeneral",
    "mathtext.fontset": "stix",
    "font.size": 22,
})

out = Output()

# ------------------------------------------------------------
# Geometry: upper semicircle (center at 0,0)
# ------------------------------------------------------------
def y_semicircle(x, R=1.0):
    return np.sqrt(np.clip(R * R - x * x, 0.0, None))

def clamp(x, lo, hi):
    return lo if x < lo else hi if x > hi else x

def poly_area_under_polyline(xs, ys):
    dx = np.diff(xs)
    return np.sum(dx * (ys[:-1] + ys[1:]) / 2.0)

# ------------------------------------------------------------
# Render
# ------------------------------------------------------------
def render_exhaustion(
    n=6,
    R=1.0,
    show_trapezoids=True,
    xmin=-1.25,
    xmax=1.25,
    fig_path=None
):
    n = int(max(2, n))
    R = float(max(0.1, R))

    xs = np.linspace(-R, R, n + 1)
    ys = y_semicircle(xs, R=R)

    true_area = 0.5 * np.pi * R * R
    approx_area = poly_area_under_polyline(xs, ys)

    err = approx_area - true_area
    rel = err / true_area if true_area != 0 else 0.0

    # convert percentage to safe string (do not put into LaTeX)
    rel_pct = f"{rel*100:+.4f}%"

    with out:
        clear_output(wait=True)

        fig, ax = plt.subplots(figsize=(12.5, 7.6), dpi=150)

        for s in ["top", "right", "bottom", "left"]:
            ax.spines[s].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])

        # Axis arrows
        ax.annotate("", xy=(xmax, 0), xytext=(xmin, 0),
                    arrowprops=dict(arrowstyle="->", lw=1.9))
        ax.annotate("", xy=(0, 1.15 * R), xytext=(0, -0.05 * R),
                    arrowprops=dict(arrowstyle="->", lw=1.9))
        ax.text(xmax, -0.10 * R, r"$x$", ha="right", va="top", fontsize=26)
        ax.text(-0.08 * R, 1.15 * R, r"$y$", ha="right", va="bottom", fontsize=26)

        # True semicircle arc
        xfine = np.linspace(-R, R, 1200)
        yfine = y_semicircle(xfine, R=R)
        ax.plot(xfine, yfine, lw=3.2)

        # Polygonal approximation
        ax.plot(xs, ys, lw=2.6)

        # Shaded approximate area
        ax.fill_between(xs, 0, ys, alpha=0.18)

        if show_trapezoids:
            for i in range(n):
                xa, xb = xs[i], xs[i + 1]
                ya, yb = ys[i], ys[i + 1]
                ax.vlines([xa, xb], [0, 0], [ya, yb], lw=1.1, alpha=0.9)

        ax.text(0.60 * R, 0.62 * R, r"$C$", fontsize=30)

        # Info box
        box = (
            rf"$n={n}$ piece approximation" + "\n"
            + rf"$A_n \approx {approx_area:.6f}$" + "\n"
            + rf"$A = \frac{{\pi R^2}}{{2}} = {true_area:.6f}$" + "\n"
            + rf"Error $A_n-A \approx {err:+.6f}$" + "\n"
            + f"Relative error ≈ {rel_pct}"
        )
        ax.text(0.05, 0.93, box, transform=ax.transAxes,
                ha="left", va="top", fontsize=17,
                bbox=dict(boxstyle="round,pad=0.55", fc="0.98", ec="0.35", lw=1.0))

        caption = (
            "Method of exhaustion (semicircle): "
            "piecewise-linear (polygonal) approximation under the arc; "
            "as $n$ increases, $A_n \\to \\pi R^2/2$."
        )
        fig.subplots_adjust(bottom=0.18)
        fig.text(0.07, 0.04, caption, fontsize=18)

        ax.set_xlim(xmin, xmax)
        ax.set_ylim(-0.20 * R, 1.25 * R)

        if fig_path:
            os.makedirs(os.path.dirname(fig_path), exist_ok=True)
            fig.savefig(fig_path, dpi=300, bbox_inches="tight")

        plt.show()

        # ---- Table + "Figure 2: ..." caption (requested) ----
        df = pd.DataFrame({
            "Metric": ["n", "R", "Approx area (A_n)", "True area (πR²/2)", "Error (A_n - A)", "Relative error"],
            "Value": [n, R, approx_area, true_area, err, rel],
        })

        # Buradaki metni istediğin gibi düzenleyebilirsin:
        table_caption = (
            f"**Figure 2:** Yarıçapı $R={R:.2f}$ olan yarım daire için, "
            f"$n={n}$ parçalı doğrusal yaklaşımın metrikleri "
            f"($A_n$, gerçek alan, hata ve bağıl hata)."
        )

        display(Markdown(table_caption))
        display(df)

# ------------------------------------------------------------
# Widgets
# ------------------------------------------------------------
slider_layout = Layout(width="340px")

n_slider = IntSlider(
    value=6, min=2, max=200, step=1, description="n",
    continuous_update=False, layout=slider_layout
)

R_slider = FloatSlider(
    value=1.0, min=0.5, max=3.0, step=0.05, description="R",
    continuous_update=False, layout=slider_layout
)

traps_chk = Checkbox(value=True, description="show trapezoid slices")

play = Play(interval=80, value=n_slider.value, min=n_slider.min, max=n_slider.max, step=1)

def _sync_play(change):
    n_slider.value = change["new"]
play.observe(_sync_play, names="value")

def _sync_slider(change):
    play.value = change["new"]
n_slider.observe(_sync_slider, names="value")

def _update(*args):
    render_exhaustion(
        n=n_slider.value,
        R=R_slider.value,
        show_trapezoids=traps_chk.value
    )

for wdg in [n_slider, R_slider, traps_chk]:
    wdg.observe(_update, names="value")

display(VBox([
    HBox([play, n_slider]),
    HBox([R_slider, traps_chk]),
    out
]))

_update()


# The method of exhaustion for the area of a parabolic segment

We apply the **method of exhaustion** to compute the area of a parabolic segment bounded by the curve

$$
y = x^2,
$$

the $x$-axis, and the vertical line $x = b$.

This region can be enclosed in a rectangle of base $b$ and altitude $b^2$.  
Archimedes discovered that the area of this parabolic segment is exactly

$$
A = \frac{b^3}{3}.
$$

We now show how this result is obtained using successive inner and outer rectangular approximations.

---

## Subdivision of the Base

We subdivide the interval $[0,b]$ into \(n\) equal parts, each of length

$$
\Delta x = \frac{b}{n}.
$$

The subdivision points are:

$$
0, \frac{b}{n}, \frac{2b}{n}, \frac{3b}{n}, \dots, \frac{(n-1)b}{n}, \frac{nb}{n} = b.
$$

A typical point is

$$
x_k = \frac{k b}{n}, \quad k = 0,1,2,\dots,n.
$$

The ordinate of the curve at this point is

$$
y_k = \left(\frac{k b}{n}\right)^2.
$$

---

## Outer Rectangles (Upper Sum)

Each rectangle has base \(\frac{b}{n}$ and height \(\left(\frac{k b}{n}\right)^2\), so its area is:

\[
\frac{b}{n} \cdot \left(\frac{k b}{n}\right)^2 = \frac{b^3}{n^3} k^2.
\]

Let \(S_n\) denote the sum of areas of all **outer rectangles**:

\[
S_n = \frac{b^3}{n^3} \left(1^2 + 2^2 + 3^2 + \cdots + n^2 \right)
\tag{1.1}
\]

---

## Inner Rectangles (Lower Sum)

Similarly, the sum of the **inner rectangles** is

\[
s_n = \frac{b^3}{n^3} \left(1^2 + 2^2 + 3^2 + \cdots + (n-1)^2 \right)
\tag{1.2}
\]

---

## Sum of Squares Formula

We use the identity

\[
1^2 + 2^2 + \cdots + n^2 = \frac{n^3}{3} + \frac{n^2}{2} + \frac{n}{6}
\tag{1.3}
\]

and

\[
1^2 + 2^2 + \cdots + (n-1)^2 = \frac{n^3}{3} - \frac{n^2}{2} + \frac{n}{6}
\tag{1.4}
\]

From these we obtain the inequalities

\[
1^2 + 2^2 + \cdots + (n-1)^2 < \frac{n^3}{3} < 1^2 + 2^2 + \cdots + n^2
\tag{1.5}
\]

Multiplying by \(\frac{b^3}{n^3}\) gives

\[
s_n < \frac{b^3}{3} < S_n
\tag{1.6}
\]

---

## Conclusion

Any number \(A\) that satisfies

\[
s_n < A < S_n
\]

for all \(n\) must be

\[
A = \frac{b^3}{3}.
\]

Therefore, the area of the parabolic segment is

\[
\boxed{A = \frac{b^3}{3}}
\]

This is the result discovered by Archimedes using the method of exhaustion.


In [8]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, VBox, Output
from IPython.display import display, clear_output

out = Output()

def f(x):
    return x**2

def draw(n=6, b=1.0):
    n = max(2, int(n))
    xs = np.linspace(0, b, 1000)
    ys = f(xs)

    xk = np.linspace(0, b, n+1)
    yk = f(xk)

    with out:
        clear_output(wait=True)
        fig, axs = plt.subplots(1, 2, figsize=(14, 5), dpi=140)

        # -------- Figure 3a: Inner rectangles
        ax = axs[0]
        ax.plot(xs, ys, lw=3)
        for i in range(n):
            ax.bar(xk[i], yk[i], width=b/n, align="edge", alpha=0.4, edgecolor="black")
        ax.set_title("Figure 3a — Approximation from below")
        ax.set_xlim(0, b)
        ax.set_ylim(0, b*b*1.1)
        ax.set_xticks([])
        ax.set_yticks([])

        # -------- Figure 3b: Outer rectangles
        ax = axs[1]
        ax.plot(xs, ys, lw=3)
        for i in range(1, n+1):
            ax.bar(xk[i-1], yk[i], width=b/n, align="edge", alpha=0.4, edgecolor="black")
        ax.set_title("Figure 3b — Approximation from above")
        ax.set_xlim(0, b)
        ax.set_ylim(0, b*b*1.1)
        ax.set_xticks([])
        ax.set_yticks([])

        plt.show()

slider = IntSlider(value=6, min=2, max=200, step=1, description="n", continuous_update=False)

def update(change):
    draw(n=slider.value)

slider.observe(update, names="value")

display(VBox([slider, out]))
draw(6)
